In [3]:
#set up Sagemaker Session
import sagemaker
sagemaker_session = sagemaker.Session()
bucket = 'meghasagemaker01'
prefix = 'sagemaker/DEMO-logistic-breast-cancer'
role = sagemaker.get_execution_role()

In [6]:
#Import data set to S3
!wget https://www.kaggle.com/senjuhashirama/breast-cancer?select=data.csv

--2020-06-08 06:07:07--  https://www.kaggle.com/senjuhashirama/breast-cancer?select=data.csv
Resolving www.kaggle.com (www.kaggle.com)... 35.244.233.98
Connecting to www.kaggle.com (www.kaggle.com)|35.244.233.98|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘breast-cancer?select=data.csv’

breast-cancer?selec     [ <=>                ]  38.28K  --.-KB/s    in 0.02s   

2020-06-08 06:07:07 (2.13 MB/s) - ‘breast-cancer?select=data.csv’ saved [39197]



In [14]:
# load data from CSV and creat independent and dependent variables
import pandas as pd
input = pd.read_csv('breast_cancer.csv')
X = input.iloc[:, :-1]
y= input.iloc[:, -1]
X.to_csv('train.csv', header=False, index=False)
validation_data.to_csv('validation.csv', header=False, index=False)

In [15]:
# Split data in train and test sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.20, random_state = 0)


In [ ]:
#transform Data
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)

In [18]:
#Train decision Tree on Training set
from sklearn.tree import DecisionTreeClassifier
classifier = DecisionTreeClassifier(random_state=0, criterion='entropy')
classifier.fit(X_train, y_train)


DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=0, splitter='best')

In [19]:
#Predict result on test set
y_pred = classifier.predict(X_test)

In [20]:
#Confusion Metrics
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, precision_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
ac = accuracy_score(y_test, y_pred)
print(ac)

[[85  2]
 [ 2 48]]
0.9708029197080292


In [21]:
#Computing accuracy with Kfold cross validation
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator=classifier, X = X_train, y = y_train, cv = 10)
accuracy_avg = accuracies.mean()*100
accuracy_deviation = accuracies.std()*100
print("Avg Accuracy : {:.2f}%".format(accuracy_avg))
print("Standard Deviation : {:.2f}%".format(accuracy_deviation))

Avg Accuracy : 95.24%
Standard Deviation : 2.49%
